In [1]:
import os
import pandas as pd
from tensorflow.python.summary.summary_iterator import summary_iterator
from tqdm import tqdm

2023-02-27 16:43:36.224845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:43:36.438020: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 16:43:36.438094: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-27 16:43:37.392490: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [8]:
def convert_tfevent(filepath):
    return pd.DataFrame([
        parse_tfevent(e) for e in summary_iterator(filepath) if len(e.summary.value)
    ])

def parse_tfevent(tfevent):
    return dict(
        wall_time=tfevent.wall_time,
        name=tfevent.summary.value[0].tag,
        step=tfevent.step,
        value=tfevent.summary.value[0],
    )

columns_order = ['wall_time', 'name', 'step', 'value']

out = []
for (root, _, filenames) in os.walk("./runs/"):
    for filename in filenames:
        if "events.out.tfevents" not in filename:
            continue
        file_full_path = os.path.join(root, filename)
        out.append(convert_tfevent(file_full_path))

# Concatenate (and sort) all partial individual dataframes
all_df = pd.concat(out)[columns_order]
all_df.reset_index(drop=True)


,wall_time,name,step,value
0,1.677500e+09,experiment_name/text_summary,0,"tag: ""experiment_name/text_summary""\ntensor {\..."
1,1.677500e+09,ds_path/text_summary,0,"tag: ""ds_path/text_summary""\ntensor {\n dtype..."
2,1.677500e+09,ds_name/text_summary,0,"tag: ""ds_name/text_summary""\ntensor {\n dtype..."
3,1.677500e+09,name_fn/text_summary,0,"tag: ""name_fn/text_summary""\ntensor {\n dtype..."
4,1.677500e+09,image_size/text_summary,0,"tag: ""image_size/text_summary""\ntensor {\n dt..."
...,...,...,...,...
7009,1.677518e+09,Loss/Train,1,"tag: ""Loss/Train""\nsimple_value: 1.25741422176..."
7010,1.677518e+09,Acc/Train,1,"tag: ""Acc/Train""\nsimple_value: 0.682600021362..."
7011,1.677518e+09,proxy_step,1,"tag: ""proxy_step""\nsimple_value: 0.0\n"
7012,1.677518e+09,Loss/Val,1,"tag: ""Loss/Val""\nsimple_value: 0.2031457573175..."


In [9]:
all_df.to_csv("collected_logs.csv")

In [20]:
all_df.iloc[8].value

tag: "change_subset_attention/text_summary"
tensor {
  dtype: DT_STRING
  tensor_shape {
    dim {
      size: 1
    }
  }
  string_val: "0.3"
}
metadata {
  plugin_data {
    plugin_name: "text"
  }
}